In [ ]:
#download the full dataset!
repo_root="/Users/miskodzamba/Dropbox/research/gits/spf/"
full_data_url='https://www.dropbox.com/scl/fi/fgppmh3q3t9m17ug4im1w/data_dec2023.zip?rlkey=j6eh34itxv2arj6ngn8imhw8v&dl=0'

!if [ ! -f  {repo_root}/full_data_v1_dl.zip ] ; then wget '{full_data_url}' -O {repo_root}/full_data_v1_dl.zip ; fi 
!if [ ! -d {repo_root}/full_data/v1 ] ; then mkdir -p {repo_root}/full_data/v1; unzip {repo_root}/full_data_v1_dl.zip -d {repo_root}/full_data/v1 ; fi 


In [ ]:
import numpy as np
import os

root_dir = repo_root + "/full_data/v1"
filenames = sorted(
    filter(
        lambda x: ".npy" in x, ["%s/%s" % (root_dir, x) for x in os.listdir(root_dir)]
    )
)
filenames = filter(lambda x: "bw_filter_" in x, filenames)

from functools import cache

# data file format
# each data file is 400,000 rows (each representing a sequential moment in time)
#   each row has 70 float values
#     time,x,y,mean,mean_trimmed,+65 values representing the signal strength in each of the 65 uniformly spaced directions
#     directions start at 0 radians and end at 2pi radians


@cache
def check_file(fn):
    try:
        m = np.memmap(fn, dtype="float32", mode="r", shape=(400000, 70))
        return not (np.abs(m).mean(axis=1) == 0).any()
    except:
        return False


filenames = list(filter(check_file, filenames))

cols = [
    "time",
    "xy[0]",
    "xy[1]",
    "mean",
    "mean_trimmed",
]

In [ ]:
print("Using %d files as input" % len(filenames))

In [ ]:
import tqdm

# read all the files and find the min and max for x (mm) and y (mm)
x_min, x_max = None, None
y_min, y_max = None, None

for filename in tqdm.tqdm(filenames):
    m = np.memmap(filename, dtype="float32", mode="r", shape=(400000, 70))
    _x_min, _x_max = m[:, 1].min(), m[:, 1].max()
    _y_min, _y_max = m[:, 2].min(), m[:, 2].max()
    if x_min is None:
        x_min = _x_min
        x_max = _x_max
        y_min = _y_min
        y_max = _y_max
    else:
        x_min = min(x_min, _x_min)
        x_max = max(x_max, _x_max)
        y_min = min(y_min, _y_min)
        y_max = max(y_max, _y_max)

In [ ]:
# split the area into a grid of bin_size
def coord_to_bin(x, y, bin_size=5):
    return int(x // bin_size), int(y // bin_size)


bin_size = 20
max_x, max_y = coord_to_bin(x_max, y_max, bin_size=bin_size)

means = np.zeros((int(max_x + 1), int(max_y + 1)))
counts = np.zeros((int(max_x + 1), int(max_y + 1)))

# find the mean estimate angle for each bin
for filename in tqdm.tqdm(filenames):
    m = np.memmap(filename, dtype="float32", mode="r", shape=(400000, 70))
    for row in m:
        x, y, mean, _mean = row[1:5]
        _x, _y = coord_to_bin(x, y, bin_size=bin_size)
        means[_x, _y] += _mean
        counts[_x, _y] += 1
means /= counts

In [ ]:
var = np.zeros((int(max_x + 1), int(max_y + 1)))

# find the variance for each bin
for filename in tqdm.tqdm(filenames):
    m = np.memmap(filename, dtype="float32", mode="r", shape=(400000, 70))
    for row in m:
        x, y, mean, _mean = row[1:5]
        _x, _y = coord_to_bin(x, y, bin_size=bin_size)
        var[_x, _y] += np.power(means[_x, _y] - _mean, 2)
var /= counts

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 2, figsize=(45, 22))

im = axs[0].imshow(
    means.T * 360 / (2 * np.pi),
    origin="upper",
    extent=(0, means.shape[0], means.shape[1], 0),
    cmap="brg",
    aspect="auto",
)
divider = make_axes_locatable(axs[0])
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(im, cax=cax, orientation="vertical")
axs[0].invert_xaxis()

im = axs[1].imshow(np.clip(var.T * 360 / (2 * np.pi), a_max=30, a_min=0))

divider = make_axes_locatable(axs[1])
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(im, cax=cax, orientation="vertical")
axs[1].invert_xaxis()

# how many ticks to put on the sides
nticks = 7
xs = np.arange(0, means.shape[0], means.shape[0] // nticks)
ys = np.arange(0, means.shape[1], means.shape[1] // nticks)

axs[0].set_title("Radio angle estimate (degrees) vs position")
axs[1].set_title("Variance (degrees) vs position")
plt.tight_layout()
plt.savefig("%dmm_res_map.png" % bin_size)

In [ ]:
import matplotlib.pyplot as plt

plt.hist(counts.reshape(-1))